In [ ]:
# Installing the required dependencies
!pip install datasets
!pip install nltk
!pip install bs4
!pip install gensim
!pip install nlpaug

In [ ]:
# Loading in the required libraries
from datasets import load_dataset
import random
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tqdm as tqdm
from bs4 import BeautifulSoup
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
# Importing the IMDb dataset from Hugging Face and creating the different splits
imdb_dataset = load_dataset("imdb")
imdb_train = imdb_dataset["train"]["text"]
imdb_train_y = np.array(imdb_dataset["train"]["label"])
imdb_test = imdb_dataset["test"]["text"]
imdb_test_y = np.array(imdb_dataset["test"]["label"])

In [ ]:
# Preprocessing our training data
stopwords_list = stopwords.words("english")
processed_reviews_train = []
for review in tqdm.tqdm(imdb_train):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_train.append(temp_update)

# Preprocessing our testing data
stopwords_list = stopwords.words("english")
processed_reviews_test = []
for review in tqdm.tqdm(imdb_test):
  raw = BeautifulSoup(review)
  html_remove = raw.get_text()
  temp = simple_preprocess(html_remove)
  temp_update = [x for x in temp if x not in stopwords_list]
  processed_reviews_test.append(temp_update)

# Defining the final word2vec model parameters
vec_size = 300
window_size = 15
model_architecture = 1
subsample = 1e-3

# Creating the document level representation using the final word2vec model for each review in the training and testing splits
w2v_model = Word2Vec(sentences = processed_reviews_train, size = vec_size, window = window_size, sg = model_architecture, sample = subsample, seed = 123)

imdb_train = np.zeros([len(processed_reviews_train), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_train))):
  word_list = []
  for word in processed_reviews_train[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_train[i] = np.mean(w2v_model.wv[word_list], axis = 0)

imdb_test = np.zeros([len(processed_reviews_test), vec_size])
for i in tqdm.tqdm(range(len(processed_reviews_test))):
  word_list = []
  for word in processed_reviews_test[i]:
    if word in w2v_model.wv.vocab:
      word_list.append(word)
      imdb_test[i] = np.mean(w2v_model.wv[word_list], axis = 0)

# Training a Naive Bayes model on the dataset
imdb_nb = GaussianNB()
imdb_nb_fit = imdb_nb.fit(imdb_train, imdb_train_y)

# Training a Logistic Regression model on the dataset
imdb_logreg = LogisticRegression(penalty = "l1", tol = 0.000001, C = 1, max_iter = 300, random_state = 123, solver = 'liblinear')
imdb_logreg_fit = imdb_logreg.fit(imdb_train, imdb_train_y)

# Training an SVM model on the dataset
imdb_svm = SGDClassifier(alpha = 0.03125, random_state = 123)
imdb_svm_fit = imdb_svm.fit(imdb_train, imdb_train_y)

# Training a Random Forests model on the dataset
imdb_rf = RandomForestClassifier(n_estimators = 300, min_samples_leaf = 2, max_features = 5, max_samples = 0.75, random_state = 123)
imdb_rf_fit = imdb_rf.fit(imdb_train, imdb_train_y)

# Using our models to obtain predictions and compute the F1-score
imdb_nb_preds = imdb_nb_fit.predict(imdb_test)
imdb_nb_f1 = f1_score(imdb_test_y, imdb_nb_preds)

imdb_logreg_preds = imdb_logreg_fit.predict(imdb_test)
imdb_logreg_f1 = f1_score(imdb_test_y, imdb_logreg_preds)

imdb_svm_preds = imdb_svm_fit.predict(imdb_test)
imdb_svm_f1 = f1_score(imdb_test_y, imdb_svm_preds)

imdb_rf_preds = imdb_rf_fit.predict(imdb_test)
imdb_rf_f1 = f1_score(imdb_test_y, imdb_rf_preds)

# Outputting the F1 score for each classifier
print(f"Naive Bayes F1 score: {imdb_nb_f1}")
print(f"Logistic Regression F1 score: {imdb_logreg_f1}")
print(f"SVM F1 score: {imdb_svm_f1}")
print(f"Random Forest F1 score: {imdb_rf_f1}")